In [1]:
import warnings
warnings.simplefilter("ignore")

In [2]:
import numpy as np

from btb.tuning.hyperparams import IntHyperParam, FloatHyperParam
from btb.tuning.tunable import Tunable
from btb.tuning.tuners import UniformTuner
from btb.tuning.tuners.gaussian_process import GPTuner

In [3]:
hyperparams = {
    'x': FloatHyperParam(-5, 5),
    'y': FloatHyperParam(-5, 5)
}
tunable = Tunable(hyperparams)

In [4]:
class FakeTuner:   
    def sample(self):
        return -5 + np.random.random() * 10
    
    def propose(self):
        return {'x': self.sample(), 'y': self.sample()}
    
    def record(self, proposal, score):
        pass

In [5]:
fake = FakeTuner()

In [6]:
%%timeit

fake.propose()

801 ns ± 12.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
%%timeit

proposal = fake.propose()
score = np.random.random()
fake.record(proposal, score)

1.15 µs ± 12.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
uf = UniformTuner(tunable)

In [9]:
%%timeit -n 100

proposal = fake.propose()
score = np.random.random()
uf.record(proposal, score)

455 µs ± 34.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit -n 100

uf.propose(allow_duplicates=True)

599 µs ± 17.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%%timeit -n 100

uf.propose(allow_duplicates=False)

598 µs ± 22.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
gp = GPTuner(tunable)

In [13]:
%%timeit -n 100

proposal = fake.propose()
score = np.random.random()
gp.record(proposal, score)

The slowest run took 18.10 times longer than the fastest. This could mean that an intermediate result is being cached.
20.6 ms ± 16 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit -n 100

gp.propose(allow_duplicates=True)

41.6 ms ± 658 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%%timeit -n 100

gp.propose(allow_duplicates=False)

46.2 ms ± 847 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
